In [1]:
import numpy as np
import pandas as pd
import pickle

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [2]:
model_filename = 'final_model.sav'

In [3]:
with open(model_filename, 'rb') as file:
    model = pickle.load(file)

In [4]:
X_valid = pd.read_csv('X_valid.csv')
y_valid = pd.read_csv('y_valid.csv')

In [5]:
# pred_proba = model.predict_proba(X_valid)

In [13]:
preds = model.predict(X_valid)

In [17]:
y_dummies = pd.get_dummies(y_valid['target'])

In [8]:
# for i in range(38):
#     if f'target_{i}' not in y_dummies.columns:
#         y_dummies[f'target_{i}'] = 0

In [9]:
columns = [f'target_{i}' for i in range(38)]

In [11]:
columns = y_dummies.columns.tolist()

In [16]:
# pred_proba = pred_proba[:, columns]
preds_dummies = pd.get_dummies(pd.DataFrame(preds))

In [19]:
precision, recall, thresholds = precision_recall_curve(y_dummies, preds_dummies)

ValueError: multilabel-indicator format is not supported

In [ ]:
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

In [23]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score, roc_auc_score

# Для каждого класса
precision = dict()
recall = dict()
thresholds = dict()
f1_score = dict()
best_precision = []
best_recall = []
best_f1_score = []
best_threshold = []
average_precision = dict()

for i in range(preds_dummies.shape[1]):
    
    precision[i], recall[i], thresholds[i] = precision_recall_curve(y_dummies.iloc[:, i], preds_dummies.iloc[:, i])
    
#     f1_score[i] = (2 * precision[i] * recall[i]) / (precision[i] + recall[i] + 1e-9)
    
#     best_index = np.argmax(f1_score[i])
    
#     best_precision.append(precision[i][best_index])
#     best_recall.append(recall[i][best_index])
#     best_f1_score.append(f1_score[i][best_index])
#     best_threshold.append(thresholds[i][best_index])
    
    average_precision[i] = average_precision_score(y_dummies.iloc[:, i], preds_dummies.iloc[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["weighted"], recall["weighted"], _ = precision_recall_curve(y_dummies.values.ravel(), pred_proba.ravel())

average_precision["weighted"] = average_precision_score(y_dummies, pred_proba, average="weighted")
print('Average precision score, micro-averaged over all classes: {0:0.2f}'.format(average_precision["weighted"]))

ValueError: Found input variables with inconsistent numbers of samples: [365112, 385396]

In [24]:
np.mean(best_precision)

/Users/macbook/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/macbook/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [ ]:
np.mean(best_recall)

In [ ]:
np.mean(best_f1_score)

In [ ]:
np.mean(best_threshold)

In [ ]:
np.mean(precision["micro"])

In [ ]:
from itertools import cycle
# setup plot details
colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])

plt.figure(figsize=(7, 8))
f_scores = np.linspace(0.2, 0.8, num=4)
lines = []
labels = []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
    plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

lines.append(l)
labels.append('iso-f1 curves')
l, = plt.plot(recall["micro"], precision["micro"], color='gold', lw=2)
lines.append(l)
labels.append('micro-average Precision-recall (area = {0:0.2f})'
              ''.format(average_precision["micro"]))

for i, color in zip(range(36), colors):
    l, = plt.plot(recall[i], precision[i], color=color, lw=2)
    lines.append(l)
    labels.append('Precision-recall for class {0} (area = {1:0.2f})'
                  ''.format(i, average_precision[i]))

fig = plt.gcf()
fig.subplots_adjust(bottom=0.25)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Extension of Precision-Recall curve to multi-class')
plt.legend(lines, labels, loc=(0,-3), prop=dict(size=14))


plt.show()

In [ ]:
pred_proba[:, [0, 1, 2, 3]]

In [31]:
pd.DataFrame(preds)[0].unique()

array([ 9, 10, 30,  8, 11, 12,  7, 13, 22, 16, 15, 37, 29, 17, 14, 31])